## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-29-16-50-00 +0000,wsj,The Trump administration said it is using an u...,https://www.wsj.com/politics/policy/trump-fore...
1,2025-08-29-16-47-01 +0000,bbc,US to stop Palestinians attending UN meeting i...,https://www.bbc.com/news/articles/cjdym32z9v7o...
2,2025-08-29-16-43-41 +0000,nypost,ESPN reporter accidentally called Coco Gauff ‘...,https://nypost.com/2025/08/29/sports/espn-repo...
3,2025-08-29-16-42-20 +0000,nypost,Chris Cuomo ratings on NewsNation down 90% com...,https://nypost.com/2025/08/29/media/chris-cuom...
4,2025-08-29-16-40-20 +0000,nypost,"Inside the gritty, hidden NYC music mecca wher...",https://nypost.com/2025/08/29/lifestyle/inside...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2313/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,56
20,new,25
275,will,14
874,shooting,11
32,up,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
267,2025-08-28-21-40-00 +0000,wsj,"Fed governor Lisa Cook sued President Trump, s...",https://www.wsj.com/economy/central-banking/li...,138
99,2025-08-29-12-34-51 +0000,bbc,Fed Governor Cook in court to block Trump from...,https://www.bbc.com/news/articles/cedvj2d5538o...,98
20,2025-08-29-16-20-11 +0000,latimes,Chabria: Newsom's cops vs. Trump's troops: A n...,https://www.latimes.com/california/story/2025-...,95
67,2025-08-29-14-04-36 +0000,nyt,Trump Asserts Expansive Power to Fire Fed Gove...,https://www.nytimes.com/2025/08/29/business/tr...,94
181,2025-08-29-04-53-57 +0000,nypost,Embattled Federal Reserve Gov. Lisa Cook hit w...,https://nypost.com/2025/08/29/us-news/federal-...,92


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
267,138,2025-08-28-21-40-00 +0000,wsj,"Fed governor Lisa Cook sued President Trump, s...",https://www.wsj.com/economy/central-banking/li...
226,67,2025-08-29-00-39-00 +0000,wsj,New Trial Ordered for Three Former Police Offi...,https://www.wsj.com/us-news/law/new-trial-orde...
44,56,2025-08-29-15-23-48 +0000,nypost,Minneapolis Catholic school shooter Robin West...,https://nypost.com/2025/08/29/us-news/minneapo...
286,51,2025-08-28-20-32-41 +0000,nypost,"UK, EU accuse Putin of ‘sabotaging peace’ afte...",https://nypost.com/2025/08/28/world-news/uk-eu...
251,44,2025-08-28-22-49-14 +0000,bbc,"'We love you, you will always be with us', say...",https://www.bbc.com/news/videos/cwy5zzvgj44o?a...
69,42,2025-08-29-14-03-22 +0000,nypost,California Gov. Newsom says crime is higher in...,https://nypost.com/2025/08/29/us-news/californ...
320,37,2025-08-28-16-56-16 +0000,latimes,ChatGPT pulled teen into a 'dark and hopeless ...,https://www.latimes.com/business/story/2025-08...
232,35,2025-08-29-00-28-26 +0000,nypost,Vance takes swipe at Mamdani — says Ugandan-bo...,https://nypost.com/2025/08/28/us-news/vance-sa...
206,35,2025-08-29-02-16-31 +0000,nypost,RFK Jr’s deputy expected to replace ousted CDC...,https://nypost.com/2025/08/28/us-news/rfk-jrs-...
51,35,2025-08-29-14-58-08 +0000,nypost,"Two more Israeli hostages, including Oct. 7 ‘h...",https://nypost.com/2025/08/29/us-news/two-more...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
